In [1]:
# source : https://towardsdatascience.com/how-to-use-the-reddit-api-in-python-5e05ddfd1e5c

import requests
import os
from dotenv import load_dotenv

load_dotenv()


# note that CLIENT_ID refers to 'personal use script' and SECRET_TOKEN to 'token'
auth = requests.auth.HTTPBasicAuth(
    os.getenv('personal_use_script'), os.getenv('SECRET_TOKEN')
)

# here we pass our login method (password), username, and password
data = {
    "grant_type": "password",
    "username": os.getenv('reddit_username'),
    "password": os.getenv('reddit_pass'),
}

# setup our header info, which gives reddit a brief description of our app
headers = {"sentiment-score": "sentiment-score/0.0.1"}

# send our request for an OAuth token
res = requests.post(
    "https://www.reddit.com/api/v1/access_token", auth=auth, data=data, headers=headers
)


print(res.json())

In [2]:
# continue from above

if res.json()['error'] == 429:
    print('come back in 5 minutes')
else:
    print(res.json())
    
    # convert response to JSON and pull access_token value
    TOKEN = res.json()["access_token"]

    # add authorization to our headers dictionary
    headers = {**headers, **{"Authorization": f"bearer {TOKEN}"}}

    # while the token is valid (~2 hours) we just add headers=headers to our requests
    requests.get("https://oauth.reddit.com/api/v1/me", headers=headers)

come back in 5 minutes


In [3]:
# get hot subreddit in r/python
res = requests.get("https://oauth.reddit.com/r/python/hot",headers=headers)

print(res.json())

JSONDecodeError: Expecting value: line 1 column 1 (char 0)

In [ ]:
# make a request for the trending posts in /r/Python, but post it in pandas dataframe

import pandas as pd

res = requests.get("https://oauth.reddit.com/r/python/hot",
                   headers=headers)

df = pd.DataFrame()  # initialize dataframe

# loop through each post retrieved from GET request
for post in res.json()['data']['children']:
    # append relevant data to dataframe
    df = df.append({
        'subreddit': post['data']['subreddit'],
        'title': post['data']['title'],
        'selftext': post['data']['selftext'],
        'upvote_ratio': post['data']['upvote_ratio'],
        'ups': post['data']['ups'],
        'downs': post['data']['downs'],
        'score': post['data']['score']
    }, ignore_index=True)

'rNxntVVdHdmaH3ykMt7wbw'